In [1]:
import argparse
import time
import warnings

from algo.LogisticCircuit import LogisticCircuit
from structure.Vtree import Vtree
from util.mnist_data import read_data_sets

FLAGS = 'None'

In [2]:
parser = argparse.ArgumentParser(description='LG')
parser.add_argument("--data_path",default="D:\\Code\\Python-Coding\\LogisticCircuit\\util\\",type=str, help="Directory for the stored input data.")

parser.add_argument("--num_classes", type=int,default=10, help="Number of classes in the classification task.")

parser.add_argument("--vtree", type=str, default="balanced.vtree", help="Path for vtree.")

parser.add_argument("--circuit",type=str,default="",help="[Optional] File path for the saved logistic circuit to load. "
                                                        "Note this circuit has to be based on the same vtree as provided in --vtree.",)

parser.add_argument("--num_structure_learning_iterations",type=int,default=5,
                    help="[Optional] Num of iterations for structure learning. Its default value is 5000.",)

parser.add_argument("--num_parameter_learning_iterations",type=int,default=15,
                    help="[Optional] Number of iterations for parameter learning after the structure is changed.""Its default value is 15.",)

parser.add_argument("--depth", type=int, default=2, help="[Optional] The depth of every split. Its default value is 2.")

parser.add_argument("--num_splits",type=int,default=3,
                    help="[Optional] The number of splits in one iteration of structure learning." "It default value is 3.",)

parser.add_argument("--percentage",type=float,default=0.2,
                    help="[Optional] The percentage of the training dataset that will be used. " "Its default value is 100%%.",)

parser.add_argument("--save_path", type=str, default="./image/", help="[Optional] File path to save the best-performing circuit.")
FLAGS = parser.parse_args(args=[])
if FLAGS.num_classes == 2:
    FLAGS.num_classes = 1
    message = (
        "It is essentially a binary classification task when num_classes is set to 2, "
        + "and hence we automatically modify it to be 1 to be better compatible with sklearn."
    )
    warnings.warn(message, stacklevel=2)
print(FLAGS)

Namespace(circuit='', data_path='D:\\Code\\Python-Coding\\LogisticCircuit\\util\\', depth=2, num_classes=10, num_parameter_learning_iterations=15, num_splits=3, num_structure_learning_iterations=5, percentage=0.2, save_path='./image/', vtree='balanced.vtree')


In [3]:
# read dataset and vtree
data = read_data_sets(FLAGS.data_path, FLAGS.percentage)
vtree = Vtree.read(FLAGS.vtree)

Extracting D:\Code\Python-Coding\LogisticCircuit\util\train-images-idx3-ubyte.gz D:\Code\Python-Coding\LogisticCircuit\util\train-labels-idx1-ubyte.gz
Extracting D:\Code\Python-Coding\LogisticCircuit\util\t10k-images-idx3-ubyte.gz D:\Code\Python-Coding\LogisticCircuit\util\t10k-labels-idx1-ubyte.gz


In [4]:
# create a logistic circuit
if FLAGS.circuit != "":
    with open(FLAGS.circuit, "r") as circuit_file:
        circuit = LogisticCircuit(vtree, FLAGS.num_classes, circuit_file=circuit_file)
        print("The saved circuit is successfully loaded.")
        data.train.features = circuit.calculate_features(data.train.images)
else:
    circuit = LogisticCircuit(vtree, FLAGS.num_classes)
    data.train.features = circuit.calculate_features(data.train.images)
    circuit.learn_parameters(data.train, 50)

D:\Software\Software_Install\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [5]:
print(f"The starting circuit has {circuit.num_parameters} parameters.")

The starting circuit has 35280 parameters.


In [6]:
data.valid.features = circuit.calculate_features(data.valid.images)
data.test.features = circuit.calculate_features(data.test.images)
valid_accuracy = circuit.calculate_accuracy(data.valid)
print(
    f"Its performance is as follows. "
    f"Training accuracy: {circuit.calculate_accuracy(data.train):.5f}\t"
    f"Valid accuracy: {valid_accuracy:.5f}\t"
    f"Test accuracy: {circuit.calculate_accuracy(data.test):.5f}"
)

print("Start structure learning.")

Its performance is as follows. Training accuracy: 0.89454	Valid accuracy: 0.85167	Test accuracy: 0.84060
Start structure learning.


In [7]:
best_accuracy = valid_accuracy
for i in range(FLAGS.num_structure_learning_iterations):
    cur_time = time.time()

    circuit.change_structure(data.train, FLAGS.depth, FLAGS.num_splits)

    data.train.features = circuit.calculate_features(data.train.images)
    data.valid.features = circuit.calculate_features(data.valid.images)
    data.test.features = circuit.calculate_features(data.test.images)

    circuit.learn_parameters(data.train, FLAGS.num_parameter_learning_iterations)

    valid_accuracy = circuit.calculate_accuracy(data.valid)
    print(
        f"Training accuracy: {circuit.calculate_accuracy(data.train):.5f}\t"
        f"Valid accuracy: {valid_accuracy:.5f}\t"
        f"Test accuracy: {circuit.calculate_accuracy(data.test):.5f}"
    )
    print(f"Num parameters: {circuit.num_parameters}\tTime spent: {(time.time() - cur_time):.2f}")

    if FLAGS.save_path != "" and (valid_accuracy > best_accuracy):
        best_accuracy = valid_accuracy
        print("Obtained a logistic circuit with higher classification accuracy. Start saving.")
        with open(FLAGS.save_path, "w") as circuit_file:
            circuit.save(circuit_file)
        print("Logistic circuit saved.")

D:\Software\Software_Install\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy: 0.89815	Valid accuracy: 0.85000	Test accuracy: 0.84100
Num parameters: 35410	Time spent: 36.57


D:\Software\Software_Install\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy: 0.90130	Valid accuracy: 0.84833	Test accuracy: 0.84110
Num parameters: 35750	Time spent: 45.08


D:\Software\Software_Install\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy: 0.90361	Valid accuracy: 0.84917	Test accuracy: 0.84130
Num parameters: 35950	Time spent: 45.09


D:\Software\Software_Install\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy: 0.90463	Valid accuracy: 0.85083	Test accuracy: 0.84150
Num parameters: 36230	Time spent: 48.22


D:\Software\Software_Install\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy: 0.90574	Valid accuracy: 0.84833	Test accuracy: 0.84190
Num parameters: 36510	Time spent: 54.14
